# MNIST solution
-- Luis Imlauer | 16/8/2023

-- Details:
- Each image is 28x28 grayscale (784pixels)
- Transform all the images, where 255 is a white pixel and 0 is a black pixel
- The input will be size 784
- 2 hidenn layers
- Output units: 10 (as there are 10 numbers 0-9)

### Importing packages

In [1]:
import tensorflow as tf
import numpy as np

import tensorflow_datasets as tfds # the mnist data is here

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Data

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info = True, as_supervised = True) # loads the data in a 2-tuple structure (input,target)

In [7]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']
# TRAIN:  60000 TEST:  10000
# Validation: 10%

num_validation_samples = tf.cast(0.1 * mnist_info.splits['train'].num_examples, tf.int64)
num_test_samples = tf.cast(mnist_info.splits['test'].num_examples, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
scaled_test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples) # create the validation dataset
train_data = shuffled_train_and_validation_data.skip(num_validation_samples) # create the training dataset without the validation data

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = scaled_test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

## Model
### Outline the model

In [11]:
input_size = 784
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape = (28,28,1)),

    tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'), # relu works great for mnist
    tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'), # 2 layers deep

    tf.keras.layers.Dense(10, activation = 'softmax'), # when creating a classifier, the activation function of the output layer must transform the values into probabilities

])

### Optimizer and loss function

In [12]:
model.compile(optimizer = 'ADAM', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

### Training the model

In [13]:
NUM_EPOCHS = 5

model.fit(train_data, epochs = NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose = 2)

Epoch 1/5
540/540 - 2s - loss: 0.3291 - accuracy: 0.9070 - val_loss: 0.1664 - val_accuracy: 0.9503 - 2s/epoch - 4ms/step
Epoch 2/5
540/540 - 1s - loss: 0.1387 - accuracy: 0.9585 - val_loss: 0.1246 - val_accuracy: 0.9633 - 1s/epoch - 2ms/step
Epoch 3/5
540/540 - 1s - loss: 0.0960 - accuracy: 0.9709 - val_loss: 0.0920 - val_accuracy: 0.9732 - 1s/epoch - 2ms/step
Epoch 4/5
540/540 - 1s - loss: 0.0734 - accuracy: 0.9779 - val_loss: 0.0728 - val_accuracy: 0.9792 - 1s/epoch - 2ms/step
Epoch 5/5
540/540 - 1s - loss: 0.0594 - accuracy: 0.9819 - val_loss: 0.0559 - val_accuracy: 0.9827 - 1s/epoch - 2ms/step


### Testing the model

In [14]:
test_loss, test_accuracy = model.evaluate(test_data)
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

1/1 [==============================] - 1s 627ms/step - loss: 0.0801 - accuracy: 0.9746
Test loss: 0.08. Test accuracy: 97.46%
